In [172]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [173]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [174]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [175]:
%%capture pwd
!pwd

In [176]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('how-will-rising-uk-energy-bills-affect-fuel-poverty-and-affordability',
 300,
 500)

# Fig 1

In [177]:
df = pd.read_excel("raw/Economics_Observatory_Charts_(1).xlsx", skiprows=1, sheet_name='Figure 1')
df.columns=['Date','Price']

In [178]:
f = "fig1_gas"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,Date,Price
0,2019-02-01,47.09
1,2019-03-01,39.31
2,2019-04-01,34.98
3,2019-05-01,31.43
4,2019-06-01,27.96


In [ ]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "Date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line = base.mark_line(color=colors["eco-dot"]).encode(
    y=alt.Y(
        "Price:Q",
        sort=[],
        axis=alt.Axis(
            gridColor=colors["eco-gray"],
            gridOpacity=0.2,
            title="Gas price (pence per therm)",
            titleX=5,
            titleY=28,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    )
)
layer1 = (
    ((line).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "w").write(readme)
layer1

# Fig 2

In [ ]:
df = pd.read_excel("raw/Economics_Observatory_Charts_(1).xlsx", skiprows=1, sheet_name='Figure 2').drop('Unnamed: 2',axis=1).dropna()
df.columns=['Date','Expenditure']
df['Date']=df['Date'].astype(str).str[:4]

In [ ]:
f = "fig2_expenditure"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

In [ ]:
base = alt.Chart(f2).encode(
    x=alt.X(
        "Date:T",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line = base.mark_line(color=colors["eco-mid-blue"]).encode(
    y=alt.Y(
        "Expenditure:Q",
        sort=[],
        axis=alt.Axis(
            gridColor=colors["eco-gray"],
            gridOpacity=0.2,
            title="Mean fuel expenditure over median household disposable income (%)",
            titleX=5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    )
).transform_filter('year(datum.Date)<2022')
line2 = base.mark_line(color=colors["eco-light-blue"],strokeDash=[5,5]).encode(
    y=alt.Y(
        "Expenditure:Q",
        )
).transform_filter('year(datum.Date)>2018')
dot1= alt.Chart(pd.DataFrame([{'x':'2022','y':8.06}])).mark_point(fill=colors["eco-dark-blue"],color=colors["eco-dark-blue"]).encode(x='x:T',y='y:Q')
dot2 = line2.mark_point(fill=colors["eco-light-blue"],color=colors["eco-light-blue"]).transform_filter('year(datum.Date)>2021')
text1=dot1.mark_text(text='Projected Ofgem tariff cap in Winter 2022-23 £2,600',align='right',color=colors['eco-dark-blue'],xOffset=-7,yOffset=-5)
text2=dot2.mark_text(text='Ofgem direct debit tariff cap in April 2022 £1,971',align='right',color=colors['eco-light-blue'],xOffset=-7,yOffset=-5)
layer1 = (
    ((line+line2+dot2+dot1+text1+text2).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1

# Fig 3

In [ ]:
df = pd.read_excel("raw/Economics_Observatory_Charts_(1).xlsx", skiprows=3, sheet_name='Figure 3')
df=df.set_index('Unnamed: 0').stack().reset_index()
df.columns=['ind','year','value']

In [ ]:
f = "fig3_fuel_poverty"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

In [ ]:
base = alt.Chart(f3).encode(
    x=alt.X(
        "year:Q",
        sort=[],
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            titleY=-15,
            format='.0f',
            titleX=207,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
        ),
    )
)
line = base.mark_line(color=colors["eco-turquiose"]).encode(
    y=alt.Y(
        "value:Q",
        sort=[],
        axis=alt.Axis(
            gridOpacity=0.2,
            gridColor=colors["eco-gray"],
            title="Households deemed Fuel Poor (%)",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            format='.0%',
            titleAlign="left",
            tickCount=7,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
        ),
    ),
    color=alt.Color('ind:N',scale=alt.Scale(range=[colors['eco-dot'],colors['eco-mid-blue'],colors['eco-turquiose']]),legend=None)
)
labels=line.mark_text(align='right',yOffset=15,xOffset=15).encode(text='ind:N').transform_joinaggregate(x='max(year)',
    groupby=["ind"]
                                                                              ).transform_filter('datum.year==datum.x')
layer1 = (
    ((line+labels).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".svg")
layer1.save("visualisation/" + f + ".png")
open("README.md", "a").write(readme)
layer1